# Career Rag chatbot
Let's take the next step in our journey! We've explored LLMs, chatbots, and RAG. Now, it's time to put them all together to create a powerful tool: a RAG chain with memory.

---
## 1.&nbsp; Installations and Settings 🛠️
Except one item, this is the same as the last notebook. The only new item is a line to download a saved version of the vector database created from Alice's Adventures in Wonderland. This saves us loading, splitting, and vectorising the book all over again.

In [ ]:
!pip3 install -qqq langchain --progress-bar off
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off
!pip3 install -qqq sentence_transformers --progress-bar off
!pip3 install -qqq faiss-gpu --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

---
## 2.&nbsp; Setting up your LLM 🧠
Here we'll add one extra parameter, `n_ctx`. This parameter controls the size of the input context. The larger the window, the more memory you're likely to use. The default of 512 is fine for most basic things, but as we are starting to retrieve articles and add them to the context window, we'll double the size to 1024. Feel free to play around with this number as your project needs.

In [1]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(model_path = "/Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/Chapter_8_generative_Ai/Model/faiss_index/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens = 2000,
               temperature = 0.1,
               top_p = 1,
               n_gpu_layers = -1,
               n_ctx = 1024)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/Chapter_8_generative_Ai/Model/faiss_index/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                

# test your MIstreal 7B LLM

In [2]:
answer_1 = llm.invoke("Write a poem about data science.")
print(answer_1)


llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =      48.21 ms /   163 runs   (    0.30 ms per token,  3381.18 tokens per second)
llama_print_timings: prompt eval time =    6641.77 ms /     8 tokens (  830.22 ms per token,     1.20 tokens per second)
llama_print_timings:        eval time =    8593.24 ms /   162 runs   (   53.04 ms per token,    18.85 tokens per second)
llama_print_timings:       total time =   15837.66 ms /   170 tokens



Data Science is the future, it's where we're all headed,
With algorithms and models, predictions are guaranteed.
We take in information from all around,
And use it to create insights that astound.

From machine learning to deep learning too,
There's nothing that data science can't do.
It helps us make decisions with ease,
And find patterns that we never knew existed.

With data visualization, we can see the world in a new light,
And discover trends that were hidden from sight.
Data science is the key to unlocking the unknown,
And helping businesses succeed and grow.

So let's embrace this future that's here to stay,
And use data science to lead the way.


## 3.&nbsp; Retrieval Augmented Generation 🔃

### 3.1.&nbsp; Find your data
Our model needs some information to work its magic! In this case, we'll be using a copy of Never Eat Alone, but feel free to swap it out for anything you like: legal documents, school textbooks, websites – the possibilities are endless!

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('/Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/Chapter_8_generative_Ai/Model/faiss_index/Never Eat Alone by Keith Ferrazzi_urdukutabkhanapk.pdf')
pages = loader.load_and_split()
documents = loader.load()

### 3.3.&nbsp; Splitting the document
Obviously, a whole book is a lot to digest. This is made easier by [splitting](https://python.langchain.com/docs/modules/data_connection/document_transformers/) the document into chunks. You can split it by paragraphs, sentences, or even individual words, depending on what you want to analyse. In Langchain, we have different tools like the RecursiveCharacterTextSplitter (say that five times fast!) that understand the structure of text and help you break it down into manageable chunks.

Check out [this website](https://chunkviz.up.railway.app/) to help visualise the splitting process.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800,
                                               chunk_overlap = 150)
docs = text_splitter.split_documents(documents)

### 3.4.&nbsp; Creating vectors with embeddings

[Embeddings](https://python.langchain.com/docs/integrations/text_embedding) are a fancy way of saying we turn words into numbers that computers can understand. Each word gets its own unique code, based on its meaning and relationship to other words. The list of numbers produced is known as a vector. Vectors allow us to compare text and find chunks that contain similar information.

Different embedding models encode words and meanings in different ways, and finding the right one can be tricky. We're using open-source models from HuggingFace, who even have a handy [leaderboard of embeddings](https://huggingface.co/spaces/mteb/leaderboard) on their website. Just browse the options and see which one speaks your language (literally!).
> As we are doing a retrieval project, click on the `Retrieval` tab of the leaderboard to see the best embeddings for retrieval tasks.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
import os

# Define the embedding model
embedding_model = 'sentence-transformers/all-MiniLM-l6-v2'

# Get the current working directory
current_folder = os.getcwd()

# Set the folder to store the embeddings cache
embeddings_folder = current_folder

# Create an instance of the HuggingFaceEmbeddings class
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
test_text = 'How to love youself?'
query_result = embeddings.embed_query(test_text)
query_result

[-0.03246622160077095,
 0.048421651124954224,
 0.10933946073055267,
 -0.007153369951993227,
 0.00035865363315679133,
 0.027365434914827347,
 0.13410817086696625,
 -0.006408788729459047,
 0.07733313739299774,
 0.004372710362076759,
 -0.0468745157122612,
 -0.07849843800067902,
 -0.006395623553544283,
 -0.008471476845443249,
 0.046537600457668304,
 -0.03503256291151047,
 -0.026914019137620926,
 0.035642609000205994,
 -0.11974657326936722,
 -0.011599737219512463,
 -0.0003129113174509257,
 -0.10261186957359314,
 -0.03776707127690315,
 -0.01161273568868637,
 -0.05223803222179413,
 0.05259639397263527,
 -0.007914486341178417,
 -0.015437936410307884,
 -0.00722751347348094,
 0.004713333677500486,
 0.09171861410140991,
 -0.016299452632665634,
 -0.015173033811151981,
 0.05441003665328026,
 -0.09058492630720139,
 0.04459471255540848,
 -0.0795581042766571,
 -0.0011238716542720795,
 0.0236909631639719,
 -0.03680482506752014,
 0.02214064449071884,
 -0.06234625354409218,
 -0.015554559417068958,
 0.042

In [7]:
characters = len(test_text)
dimensions = len(query_result)
print(f"the {characters}' character sentence was transformed into a {dimensions} dimension vector")

the 20' character sentence was transformed into a 384 dimension vector


Embedding vectors have a fixed length, meaning each vector produced by this specific embedding will always have 384 dimensions. Choosing the appropriate embedding size involves a trade-off between accuracy and computational efficiency. Larger embeddings capture more semantic information but require more memory and processing power. Start with the provided MiniLM embedding as a baseline and experiment with different sizes to find the optimal balance for your needs.

### 3.5.&nbsp; Creating a vector database
Imagine a library where books aren't just filed alphabetically, but also by their themes, characters, and emotions. That's the magic of vector databases: they unlock information beyond keywords, connecting ideas in unexpected ways.

In [8]:
from langchain.vectorstores import FAISS
vector_db2 = FAISS.from_documents(docs, embeddings)

In [9]:
vector_db2.save_local('/Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/faiss_index2')

You can also search your database to see which vectors are close to your input.

In [10]:
vector_db2.similarity_search('how to grow your network?')

[Document(page_content='Never Eat Alone 97 \na run. As a makeshift staff meeting, I\'ll occasionally ask a few \nemployees to share a car ride with me to the airport. I figure out \nways to as much as triple my active working day through such \nmultitasking. And, in the process, I\'m connecting people from dif\xad\nferent parts of my "community." \nThe more new connections you establish, the more opportuni\xad\nties you\'ll have to make even more new connections. As Robert \nMetcalfe, the inventor of Ethernet, says: The value of a network \ngrows proportional to the square of the number of its users. In the \ncase of the Internet, every new computer, every new server, and \nevery new user added expands the possibilities for everyone else \nwho\'s already there. The same principle holds true in growing', metadata={'source': '/Users/sadiakhanrupa/Bootcamp Main Phase/Chapter_8_generative_Ai/Chapter_8_generative_Ai/Model/faiss_index/Never Eat Alone by Keith Ferrazzi_urdukutabkhanapk.pdf', 

In [11]:
#retriever 
retriever = vector_db2.as_retriever(search_kwargs={'k':2})

In [12]:
#creating memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key='chat_history',
                                  return_messages=True,
                                  output_key='answer')

In [13]:
#creating Prompt
from langchain import PromptTemplate
# prompt
template = """
<s> [INST]
You are polite and professional question-answering AI assistant. You must provide a helpful response to the user.

In your response, PLEASE ALWAYS:
  (0) Be a career Analyst: give precise answers how one person can develop his career
  (1) Provide examples of other people: how growing network has been helpful for them
  (2) If the context enables you to answer the question, write a detailed, helpful, and easily understandable answer. If you can't find the answer, respond with an explanation, starting with: "I couldn't find the answer in the information I have access to".
  (3) Do not make answers too boring, add some humor with your answer
[/INST]
[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
[INST]
Helpful Answer:
[INST]
"""

prompt = PromptTemplate(template = template,
                        input_variables = ['context', 'question'])

In [14]:
#creating chain
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever=retriever,
                                              memory=memory,
                                              return_source_documents=True,
                                              combine_docs_chain_kwargs={'prompt':prompt})

In [15]:
chain.invoke('how to make friends?')

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =     107.21 ms /   378 runs   (    0.28 ms per token,  3525.72 tokens per second)
llama_print_timings: prompt eval time =   22101.90 ms /   643 tokens (   34.37 ms per token,    29.09 tokens per second)
llama_print_timings:        eval time =   21942.85 ms /   377 runs   (   58.20 ms per token,    17.18 tokens per second)
llama_print_timings:       total time =   45961.06 ms /  1020 tokens


{'question': 'how to make friends?',
 'chat_history': [HumanMessage(content='how to make friends?'),
  AIMessage(content="To make friends with little or no fear, you can start by learning to speak confidently and effectively. This will help you overcome shyness and feel more comfortable in social situations. You can also try joining groups or clubs that share your interests, such as a book club or a hobby group. This will give you an opportunity to meet people who are passionate about the same things as you and make it easier to start conversations. Additionally, you can ask for help from others who have experience making friends, such as colleagues or friends of friends. They may be able to offer advice and support as you work on building your social skills.\n\nHere are some examples of how growing a network has been helpful for others:\n\n* John was feeling lonely and isolated at work, but he joined a book club and met several people who shared his love of reading. Over time, he made

In [16]:
print(chain.invoke('how to make friends?')['answer'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =       4.86 ms /    13 runs   (    0.37 ms per token,  2672.70 tokens per second)
llama_print_timings: prompt eval time =   12471.70 ms /   439 tokens (   28.41 ms per token,    35.20 tokens per second)
llama_print_timings:        eval time =     663.92 ms /    12 runs   (   55.33 ms per token,    18.07 tokens per second)
llama_print_timings:       total time =   13385.66 ms /   451 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =      81.39 ms /   227 runs   (    0.36 ms per token,  2788.94 tokens per second)
llama_print_timings: prompt eval time =   18574.25 ms /   653 tokens (   28.44 ms per token,    35.16 tokens per second)
llama_print_timings:        eval time =   12986.82 ms /   226 runs   (   57.46 ms per token,    17.40 tokens per second)
llama_print_timings:       to

To improve your social skills, you can start by learning how to speak confidently in different situations. This can be done through various educational organizations that offer courses on public speaking and overcoming shyness. Additionally, observing and learning from others who are skilled at making small talk and building relationships can also help you develop your own social skills.

For example, the book "Never Eat Alone" by Keith Ferrazzi highlights the importance of face-to-face communication in building relationships. The author emphasizes that while digital communication may be efficient, it is not effective when it comes to making friends. He suggests that conversation is an acquired skill and can be learned with determination and proper information.

Another example is the CEOs who take pride in their ability to negotiate social situations with relative ease. These individuals have likely developed their social skills through years of practice and observation, and they can 

In [17]:
print(chain.invoke('how I can get a job as Data Scientist in germany')['answer'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =       5.17 ms /    14 runs   (    0.37 ms per token,  2706.88 tokens per second)
llama_print_timings: prompt eval time =   19548.63 ms /   686 tokens (   28.50 ms per token,    35.09 tokens per second)
llama_print_timings:        eval time =     739.13 ms /    13 runs   (   56.86 ms per token,    17.59 tokens per second)
llama_print_timings:       total time =   20713.55 ms /   699 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6641.83 ms
llama_print_timings:      sample time =     135.41 ms /   360 runs   (    0.38 ms per token,  2658.55 tokens per second)
llama_print_timings: prompt eval time =   18795.86 ms /   658 tokens (   28.57 ms per token,    35.01 tokens per second)
llama_print_timings:        eval time =   20735.70 ms /   359 runs   (   57.76 ms per token,    17.31 tokens per second)
llama_print_timings:       to

As a data scientist, finding job opportunities in Germany can be challenging, but with the right approach, it's possible to land your dream job. Here are some tips to help you get started:

1. Networking: Building a strong network is crucial for finding job opportunities as a data scientist. Attend industry events, join professional organizations, and connect with other professionals in the field on LinkedIn. You can also reach out to recruiters and hiring managers directly to learn more about open positions.
2. Online Job Boards: There are many online job boards that specialize in data science jobs in Germany. Some popular options include Glassdoor, Indeed, and LinkedIn Jobs. These platforms allow you to search for job postings based on your skills and experience.
3. Personal Branding: Developing a strong personal brand can help you stand out from other candidates and increase your chances of landing a job. This can include creating a professional website or blog, publishing articles 